chemical compliance data analysis

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from thefuzz import fuzz, process

import data 

In [3]:
data = pd.read_excel(r"C:\Users\Welcome\Downloads\customer_analysis_data.xlsx")
data.head()


,Request_ID,Customer_ship_code,Customer_no,Customer_name,Request_raised_date,Manufacturer_part_no,Manufacturer_name,Manufacturer_code,Request_Type,Document_submitted_date
0,R20240515154046,FUTC,KY111600,"KYODEN PRECISION CO., LTD.",2024-05-15 15:40:46,SJ-435107RS,CUI DEVICES,C1D,Materials Declaration Sheet,2024-05-15 15:40:46
1,R20240515154046,FUTC,KY111600,"KYODEN PRECISION CO., LTD.",2024-05-15 15:40:46,SJ-435107RS,CUI DEVICES,C1D,PFAS Declaration,2024-06-10 17:09:17
2,R20240516232215,FUTA,HK103500,HSGT,2024-05-16 23:22:15,IRF7855TRPBF,INFINEON,INF,Materials Declaration Sheet,2024-05-16 23:22:15
3,R20240516232215,FUTA,HK103500,HSGT,2024-05-16 23:22:15,IRF7855TRPBF,INFINEON,INF,ROHS(EU) 2015/863,2024-05-16 23:22:15
4,R20240516232215,FUTA,HK103500,HSGT,2024-05-16 23:22:15,IRF7855TRPBF,INFINEON,INF,SGS Test Report - Including ROHS Phthalates,2024-05-16 23:22:15


Data cleaning time

In [4]:
data['Request_raised_date'] = pd.to_datetime(data['Request_raised_date']).dt.date
data['Document_submitted_date'] = pd.to_datetime(data['Document_submitted_date']).dt.date
data.head()

,Request_ID,Customer_ship_code,Customer_no,Customer_name,Request_raised_date,Manufacturer_part_no,Manufacturer_name,Manufacturer_code,Request_Type,Document_submitted_date
0,R20240515154046,FUTC,KY111600,"KYODEN PRECISION CO., LTD.",2024-05-15,SJ-435107RS,CUI DEVICES,C1D,Materials Declaration Sheet,2024-05-15
1,R20240515154046,FUTC,KY111600,"KYODEN PRECISION CO., LTD.",2024-05-15,SJ-435107RS,CUI DEVICES,C1D,PFAS Declaration,2024-06-10
2,R20240516232215,FUTA,HK103500,HSGT,2024-05-16,IRF7855TRPBF,INFINEON,INF,Materials Declaration Sheet,2024-05-16
3,R20240516232215,FUTA,HK103500,HSGT,2024-05-16,IRF7855TRPBF,INFINEON,INF,ROHS(EU) 2015/863,2024-05-16
4,R20240516232215,FUTA,HK103500,HSGT,2024-05-16,IRF7855TRPBF,INFINEON,INF,SGS Test Report - Including ROHS Phthalates,2024-05-16


In [5]:
data.shape

(24073, 10)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24073 entries, 0 to 24072
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Request_ID               24073 non-null  object
 1   Customer_ship_code       24073 non-null  object
 2   Customer_no              24073 non-null  object
 3   Customer_name            24073 non-null  object
 4   Request_raised_date      24073 non-null  object
 5   Manufacturer_part_no     24073 non-null  object
 6   Manufacturer_name        24073 non-null  object
 7   Manufacturer_code        24073 non-null  object
 8   Request_Type             24073 non-null  object
 9   Document_submitted_date  24073 non-null  object
dtypes: object(10)
memory usage: 1.8+ MB


data cleaning manufacturing name

In [7]:

# Initialize an empty dictionary to store standardized names
standardized_dict = {}

# Function to standardize manufacturer names
def standardize_manufacturer(name):
    # Split on space and slash
    first_word = name.split(' ')[0].split('/')[0].upper()  # Take the first word and convert to uppercase
    # If the first word is already in the standardized dict, use its standardized name
    if first_word in standardized_dict:
        return standardized_dict[first_word]
    else:
        # If not present, add it to the dict with itself as the value
        standardized_dict[first_word] = first_word
        return first_word

# Apply the standardization function to the 'manufacturer_name' column
data['standardized_manufacturer_name'] = data['Manufacturer_name'].apply(standardize_manufacturer)

# Display the updated DataFrame
data.head()


,Request_ID,Customer_ship_code,Customer_no,Customer_name,Request_raised_date,Manufacturer_part_no,Manufacturer_name,Manufacturer_code,Request_Type,Document_submitted_date,standardized_manufacturer_name
0,R20240515154046,FUTC,KY111600,"KYODEN PRECISION CO., LTD.",2024-05-15,SJ-435107RS,CUI DEVICES,C1D,Materials Declaration Sheet,2024-05-15,CUI
1,R20240515154046,FUTC,KY111600,"KYODEN PRECISION CO., LTD.",2024-05-15,SJ-435107RS,CUI DEVICES,C1D,PFAS Declaration,2024-06-10,CUI
2,R20240516232215,FUTA,HK103500,HSGT,2024-05-16,IRF7855TRPBF,INFINEON,INF,Materials Declaration Sheet,2024-05-16,INFINEON
3,R20240516232215,FUTA,HK103500,HSGT,2024-05-16,IRF7855TRPBF,INFINEON,INF,ROHS(EU) 2015/863,2024-05-16,INFINEON
4,R20240516232215,FUTA,HK103500,HSGT,2024-05-16,IRF7855TRPBF,INFINEON,INF,SGS Test Report - Including ROHS Phthalates,2024-05-16,INFINEON


In [8]:
uni_manu=data['Manufacturer_name'].nunique()
print(uni_manu)

uni_stand_manu=data['standardized_manufacturer_name'].nunique()
print(uni_stand_manu)

348
286


extract this cleaned data

In [9]:
data.to_csv("cleaned_organized_data.csv",index=False)

connect my sql

In [10]:
import mysql.connector

# Establish a connection to the MySQL database
connection = mysql.connector.connect(
    host='localhost',          # Change to your host if different
    user='root',      # Your MySQL username
    password='Hari@123',  # Your MySQL password
    database='customerdataanalysis'   # Your database name
)

# Create a cursor object
cursor = connection.cursor()


In [11]:
insert_query = """
INSERT INTO customerdata (Request_ID, Customer_ship_code, Customer_no, Customer_name, 
                              Request_raised_date, Manufacturer_part_no, Manufacturer_name, 
                              Manufacturer_code, Request_Type, Document_submitted_date, 
                              standardized_manufacturer_name)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Loop through the DataFrame and insert each row
for i, row in data.iterrows():
    cursor.execute(insert_query, tuple(row))

# Commit the transaction to save the changes
connection.commit()

In [13]:
def query_database(query):
    cursor.execute(query)
    result = cursor.fetchall()
    columns = cursor.column_names
    return pd.DataFrame(result, columns=columns)

# Example query to select all data from your table
query = "SELECT * FROM customerdata"
data = query_database(query)

print(data)

            Request_ID Customer_ship_code Customer_no  \
0      R20240515154046               FUTC    KY111600   
1      R20240515154046               FUTC    KY111600   
2      R20240516232215               FUTA    HK103500   
3      R20240516232215               FUTA    HK103500   
4      R20240516232215               FUTA    HK103500   
...                ...                ...         ...   
24068  R20241019000316               FUTA    PS550000   
24069  R20241019000316               FUTA    PS550000   
24070  R20241019000316               FUTA    PS550000   
24071  R20241019000316               FUTA    PS550000   
24072  R20241019000316               FUTA    PS550000   

                           Customer_name Request_raised_date  \
0      KYODEN PRECISION CO., LTD.                 2024-05-15   
1      KYODEN PRECISION CO., LTD.                 2024-05-15   
2      HSGT                                       2024-05-16   
3      HSGT                                       2024-05-1

In [16]:
query = "SELECT count(distinct Request_ID) as total_request_created FROM customerdata"
data = query_database(query)

print(data)

   total_request_created
0                   1467


In [18]:
query = "SELECT count(distinct Customer_name) as total_no_of_customer FROM customerdata"
data = query_database(query)

print(data)

   total_no_of_customer
0                   754


In [20]:
query = "SELECT Customer_ship_code, case when Customer_ship_code='FUTC' THEN 'MIDDLE EAST AND ASIA' when Customer_ship_code='FUTA' THEN 'AMERICA' when Customer_ship_code='FUTE' THEN 'EUROPE' when Customer_ship_code='FUTI' THEN 'CANADA' END as country,  count(Customer_no) as total_customer FROM customerdata group by Customer_ship_code"
data = query_database(query)

print(data)

  Customer_ship_code               country  total_customer
0               FUTC  MIDDLE EAST AND ASIA           11278
1               FUTA               AMERICA            6123
2               FUTE                EUROPE            4827
3               FUTI                CANADA            1845


In [21]:
query = "select monthname(Request_raised_date) , count(distinct Request_ID) from customerdata group by monthname(Request_raised_date)"
data = query_database(query)

print(data)

  monthname(Request_raised_date)  count(distinct Request_ID)
0                          April                         122
1                         August                         229
2                           July                         214
3                           June                         255
4                            May                         219
5                        October                         211
6                      September                         217


In [23]:
query = "select Customer_name, group_concat(distinct Request_Type) as total_request_types from customerdata group by Customer_name"
data = query_database(query)

print(data)

                         Customer_name  \
0    3M COMPANY                          
1    ABB AUTOMATION INC                  
2    ABB E-MOBILITY SPA                  
3    ABB ELECTRIFICATION CANADA INC      
4    ABB INC                             
..                                 ...   
749  YONGTECHS ELECTRIC CO., LTD.        
750  ZHEJIANG RAYSOAR ELECTRONIC TECH    
751  ZHU HAI SIRTEC IMAGE & TECHNOLOGY   
752  ZHUHAI COSMX POWER SUPPLY CO.,LTD   
753  ZHUHAI FTZ OPLINK COMMUNICATIONS    

                                   total_request_types  
0    Russian Federation Iron and Steel Ban (EU 833/...  
1    Battery Directive (EU) 2023/1542,Persistent Or...  
2    CFSI CMRT 6.3 (2023),PFAS Declaration,REACH 30...  
3                                     PFAS Declaration  
4    Battery Directive (EU) 2023/1542,CA Prop 65 De...  
..                                                 ...  
749                               CFSI CMRT 6.3 (2023)  
750  China ROHS 2,REACH 30 - 240 SVHC C

In [28]:
query = "select standardized_manufacturer_name,count(distinct Manufacturer_part_no) as total_Mpn from customerdata group by standardized_manufacturer_name order by total_Mpn desc limit 10"
data = query_database(query)

print(data)

  standardized_manufacturer_name  total_Mpn
0                         VISHAY       1081
1                          YAGEO        997
2                             TE        855
3                         ONSEMI        660
4                     LITTELFUSE        601
5                      PANASONIC        586
6                            AVX        551
7                             ST        505
8                         MURATA        476
9                         DIODES        438


In [29]:
query = "select Request_Type,count(distinct Manufacturer_part_no) as total_Mpn from customerdata group by Request_Type order by total_Mpn desc"
data = query_database(query)

print(data)

                                         Request_Type  total_Mpn
0                                    PFAS Declaration       4164
1                                     CMRT 6.4 (2024)       3755
2                   REACH 30 - 240 SVHC Certification       1871
3                                 REACH 31 (SVHC 241)       1803
4     EMRT (Extended Minerals Reporting Template) 1.3       1765
5                                   ROHS(EU) 2015/863       1734
6                       CA Prop 65 Declaration (2023)       1524
7                                   TSCA Restrictions        723
8                         Materials Declaration Sheet        681
9                                                IMDS        655
10                               CFSI CMRT 6.3 (2023)        626
11                   Special Request - See Attachment        479
12  Russian Federation Iron and Steel Ban (EU 833/...        343
13  Persistent Organic Pollutants (EC No. 850 / 2004)        339
14                       

In [31]:
query = "select Request_Type,count(distinct standardized_manufacturer_name) as total_mfr from customerdata group by Request_Type order by total_mfr desc"
data = query_database(query)

print(data)

                                         Request_Type  total_mfr
0                                    PFAS Declaration        196
1                                   ROHS(EU) 2015/863        172
2                                     CMRT 6.4 (2024)        169
3                   REACH 30 - 240 SVHC Certification        168
4                                 REACH 31 (SVHC 241)        143
5                       CA Prop 65 Declaration (2023)        129
6                                CFSI CMRT 6.3 (2023)        118
7     EMRT (Extended Minerals Reporting Template) 1.3        117
8                         Materials Declaration Sheet        114
9                                   TSCA Restrictions         94
10                   Special Request - See Attachment         76
11  Persistent Organic Pollutants (EC No. 850 / 2004)         61
12                                               IMDS         60
13  Russian Federation Iron and Steel Ban (EU 833/...         53
14                   Batt

In [33]:
query = "select Request_Type,count(distinct Customer_name) as total_customer from customerdata group by Request_Type order by total_customer desc"
data = query_database(query)

print(data)

                                         Request_Type  total_customer
0                   REACH 30 - 240 SVHC Certification             193
1                                   ROHS(EU) 2015/863             187
2                                 REACH 31 (SVHC 241)             157
3                         Materials Declaration Sheet             156
4                                                IMDS             121
5                                     CMRT 6.4 (2024)             116
6                                    PFAS Declaration             107
7                                CFSI CMRT 6.3 (2023)              71
8                       CA Prop 65 Declaration (2023)              70
9                                   TSCA Restrictions              65
10                  MSDS (Material Safety Data Sheet)              49
11    EMRT (Extended Minerals Reporting Template) 1.3              48
12                   Special Request - See Attachment              47
13        SGS Test R

add new column to find day difference

In [36]:
customer_data = pd.read_csv(r"C:\Users\Welcome\Desktop\data project\cleaned_organized_data.csv")
customer_data.head()


,Request_ID,Customer_ship_code,Customer_no,Customer_name,Request_raised_date,Manufacturer_part_no,Manufacturer_name,Manufacturer_code,Request_Type,Document_submitted_date,standardized_manufacturer_name
0,R20240515154046,FUTC,KY111600,"KYODEN PRECISION CO., LTD.",2024-05-15,SJ-435107RS,CUI DEVICES,C1D,Materials Declaration Sheet,2024-05-15,CUI
1,R20240515154046,FUTC,KY111600,"KYODEN PRECISION CO., LTD.",2024-05-15,SJ-435107RS,CUI DEVICES,C1D,PFAS Declaration,2024-06-10,CUI
2,R20240516232215,FUTA,HK103500,HSGT,2024-05-16,IRF7855TRPBF,INFINEON,INF,Materials Declaration Sheet,2024-05-16,INFINEON
3,R20240516232215,FUTA,HK103500,HSGT,2024-05-16,IRF7855TRPBF,INFINEON,INF,ROHS(EU) 2015/863,2024-05-16,INFINEON
4,R20240516232215,FUTA,HK103500,HSGT,2024-05-16,IRF7855TRPBF,INFINEON,INF,SGS Test Report - Including ROHS Phthalates,2024-05-16,INFINEON


In [39]:
customer_data['Request_raised_date'] = pd.to_datetime(customer_data['Request_raised_date'])
customer_data['Document_submitted_date'] = pd.to_datetime(customer_data['Document_submitted_date'])

# Calculate the difference in days
customer_data['day_difference'] = (customer_data['Document_submitted_date'] - customer_data['Request_raised_date']).dt.days

# Display the updated DataFrame
customer_data.head()

,Request_ID,Customer_ship_code,Customer_no,Customer_name,Request_raised_date,Manufacturer_part_no,Manufacturer_name,Manufacturer_code,Request_Type,Document_submitted_date,standardized_manufacturer_name,day_difference
0,R20240515154046,FUTC,KY111600,"KYODEN PRECISION CO., LTD.",2024-05-15,SJ-435107RS,CUI DEVICES,C1D,Materials Declaration Sheet,2024-05-15,CUI,0
1,R20240515154046,FUTC,KY111600,"KYODEN PRECISION CO., LTD.",2024-05-15,SJ-435107RS,CUI DEVICES,C1D,PFAS Declaration,2024-06-10,CUI,26
2,R20240516232215,FUTA,HK103500,HSGT,2024-05-16,IRF7855TRPBF,INFINEON,INF,Materials Declaration Sheet,2024-05-16,INFINEON,0
3,R20240516232215,FUTA,HK103500,HSGT,2024-05-16,IRF7855TRPBF,INFINEON,INF,ROHS(EU) 2015/863,2024-05-16,INFINEON,0
4,R20240516232215,FUTA,HK103500,HSGT,2024-05-16,IRF7855TRPBF,INFINEON,INF,SGS Test Report - Including ROHS Phthalates,2024-05-16,INFINEON,0


In [46]:
query = "SELECT a.Request_ID, AVG(DATEDIFF(a.Document_submitted_date, a.Request_raised_date)) AS avg_day_difference FROM (select * from customerdata where length(Request_ID)>8) a GROUP BY a.Request_ID order by avg_day_difference desc"
data = query_database(query)

print(data)

           Request_ID avg_day_difference
0     R20240528153442           141.0000
1     R20240530112114           137.0000
2     R20240529111258           120.0000
3     R20240618100914           119.0000
4     R20240614095832           115.0000
...               ...                ...
1099  R20240702153647            -4.0000
1100  R20240705121608            -5.0000
1101  R20240701182443            -5.5000
1102  R20240702180240            -6.0000
1103  R20240930115217            -6.0000

[1104 rows x 2 columns]


In [47]:
query = "with cte as (SELECT a.Request_ID, AVG(DATEDIFF(a.Document_submitted_date, a.Request_raised_date)) AS avg_day_difference FROM (select * from customerdata where length(Request_ID)>8) a GROUP BY a.Request_ID order by avg_day_difference desc) select avg(cte.avg_day_difference) from cte"
data = query_database(query)

print(data)

  avg(cte.avg_day_difference)
0                  8.27591178
